In [3]:
import subprocess
import os
import signal
from glob import glob

# --------- CONFIGURATION ---------
data_yaml = "data.yaml"              # Ensure this is placed inside yolov5/
weights = "yolov5n.pt"               # Use nano model for lightweight prototype
epochs = 10                          # For quick iteration
batch_size = 16
img_size = 640
project_dir = "runs/train"           # Save training logs here
exp_name = "box_proto"               # Your experiment name

# Path to checkpoint to resume from (None to auto detect)
resume_checkpoint = None

# --------- VALIDATION ---------
if not os.path.exists(data_yaml):
    raise FileNotFoundError(f" Could not find data.yaml at: {data_yaml}")
else:
    print(f" Found data.yaml at: {data_yaml}")

# --------- CHECK FOR EXISTING CHECKPOINT ---------
checkpoint_dir = os.path.join(project_dir, exp_name, "weights")
if os.path.isdir(checkpoint_dir):
    checkpoints = glob(os.path.join(checkpoint_dir, "*.pt"))
    if checkpoints:
        latest_ckpt = max(checkpoints, key=os.path.getctime)
        print(f" Found checkpoint to resume: {latest_ckpt}")
        resume_checkpoint = latest_ckpt

# --------- BUILD COMMAND ---------
command = [
    "python", "train.py",
    "--data", data_yaml,
    "--epochs", str(epochs),
    "--batch-size", str(batch_size),
    "--img", str(img_size),
    "--project", project_dir,
    "--name", exp_name,
    "--exist-ok"
]

if resume_checkpoint:
    print(f" Resuming from checkpoint: {resume_checkpoint}")
    command.extend(["--weights", resume_checkpoint])
elif weights:
    print(f" Starting from pretrained weights: {weights}")
    command.extend(["--weights", weights])
else:
    print(" Training from scratch (no pretrained weights).")
    command.extend(["--weights", ""])

# --------- RUN TRAINING ---------
print(f"\n Running training command:\n{' '.join(command)}\n")

process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

try:
    for line in process.stdout:
        print(line, end='')  # live output in notebook/terminal
except KeyboardInterrupt:
    print("\n Training interrupted by user!")
    process.send_signal(signal.SIGINT)
    process.wait()

 Found data.yaml at: data.yaml
 Starting from pretrained weights: yolov5n.pt

 Running training command:
python train.py --data data.yaml --epochs 10 --batch-size 16 --img 640 --project runs/train --name box_proto --exist-ok --weights yolov5n.pt

train: weights=yolov5n.pt, cfg=, data=data.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=10, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data\hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=box_proto, exist_ok=True, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 
YOLOv5  v7